# Road Traffic Accidents in Switzerland

Our project goal is to scrap all traffic accidents from the accidents map from http://map.donneesaccidents.ch/

## Data scraping strategy

Accessing http://map.donneesaccidents.ch/, wich redirects to : <br>
https://map.geo.admin.ch/?topic=vu&lang=fr&bgLayer=ch.swisstopo.pixelkarte-grau&layers=ch.astra.unfaelle-personenschaeden_alle&layers_timestamp=&catalogNodes=1318


Postman parses the following parameters : 
<code>
topic:vu
lang:en
bgLayer:ch.swisstopo.pixelkarte-grau
layers:ch.astra.unfaelle-personenschaeden_alle
layers_timestamp:
catalogNodes:1318
</code>

The most important one is layers:ch.astra.unfaelle-personenschaeden_alle.<br>
It is the layer that contains all the geo-information dots on "Accidents with personal injury" which is the selected data layer.
<img src="layer_selector.png">

Selection all kinds of accidents returns the following :<br>
<img src="layer_selector_all.png">
with layer parameters :<br>
layers:<br>
    &nbsp;ch.astra.unfaelle-personenschaeden_alle,<br>
    &nbsp;ch.astra.unfaelle-personenschaeden_getoetete,<br>
    &nbsp;ch.astra.unfaelle-personenschaeden_fussgaenger,<br>
    &nbsp;ch.astra.unfaelle-personenschaeden_fahrraeder,<br>
    &nbsp;ch.astra.unfaelle-personenschaeden_motorraeder<br>
layers_timestamp:,,,,<br>

Now we want every data for each layer. By selecting a dot on the map, it queries the related data to the server.
What we want to do is selecting all the entries in the map to retrieves all data. This is done by ctrl clicking the whole area.

This makes a query for each "layers" parameter :
<code>
geometry:443999.04209536605,39001.6733318335,870499.0420953662,303001.67333183356
geometryFormat:geojson
geometryType:esriGeometryEnvelope
imageDisplay:1536,759,96
lang:en
layers:all:<i>LAYER_PARAM</i>
mapExtent:269999.04209536605,9501.673331833561,1037999.042095366,389001.67333183356
returnGeometry:true
tolerance:5
</code><br>
But doesn't select all dots on map, so let's try the "load more results" button on a 'accidetns with fatalities' layer, we get :
<code>
geometry:443999.04209536605,39001.6733318335,870499.0420953662,303001.67333183356
geometryFormat:geojson
geometryType:esriGeometryEnvelope
imageDisplay:1536,759,96
lang:en
layers:all:ch.astra.unfaelle-personenschaeden_getoetete
mapExtent:136199.04209536605,-28148.32666816644,1134599.042095366,465201.67333183356
<b>offset:200</b>
returnGeometry:true
tolerance:5
</code>
Pressing load more until no more possible give offset=1200 (for a total of 1337 objects) i.e it loads data entries 200 by 200

## JSON Data scraping

In [1]:
import requests
import json

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from Scripts.helpers import *
from Scripts.plots import *


import pprint
#from bs4 import BeautifulSoup

In [2]:
pp = pprint.PrettyPrinter(indent=4)

In [3]:
#import raw data
data = import_data(all_data = False)

Processing layer : ch.astra.unfaelle-personenschaeden_alle
Break in loop while
Layer processed : 201 records

Processing layer : ch.astra.unfaelle-personenschaeden_getoetete
Break in loop while
Layer processed : 201 records

Processing layer : ch.astra.unfaelle-personenschaeden_fussgaenger
Break in loop while
Layer processed : 201 records

Processing layer : ch.astra.unfaelle-personenschaeden_fahrraeder
Break in loop while
Layer processed : 201 records

Processing layer : ch.astra.unfaelle-personenschaeden_motorraeder
Break in loop while
Layer processed : 201 records

Whole dataset processed : 1005 records



In [4]:
#translate data from german
json_data_preprocessed = preprocess_data(data)

In [5]:
print("Data entry example after clean and reformat:\n")
json_data_preprocessed[0]

Data entry example after clean and reformat:



{'accidentday_fr': 'vendredi / 13h-14h / juin 2013',
 'accidenttype_fr': 'dérapage ou perte de maîtrise',
 'accidenttypecode': 0,
 'accidentyear': 2013,
 'bbox': [566407.0, 114552.0, 566407.0, 114552.0],
 'canton': 'VS',
 'coordinates': [566407.0, 114552.0],
 'featureId': 'E1229FF577AA00D8E0430A8394270E90',
 'fsocommunecode': '6217',
 'geometryType': 'Feature',
 'id': 'E1229FF577AA00D8E0430A8394270E90',
 'label': 'Schleuder- oder Selbstunfall',
 'layerBodId': 'ch.astra.unfaelle-personenschaeden_alle',
 'layerName': 'Accidents avec dommages corporels',
 'roadtype_fr': 'route secondaire',
 'roadtypecode': 433,
 'severitycategory_fr': 'accident avec blessés graves',
 'severitycategorycode': 'USV',
 'type': 'Feature'}

In [6]:
df = pd.DataFrame.from_dict(json_data_preprocessed)
df.set_index('id', inplace=True)
df.sample(5)

,accidentday_fr,accidenttype_fr,accidenttypecode,accidentyear,bbox,canton,coordinates,featureId,fsocommunecode,geometryType,label,layerBodId,layerName,roadtype_fr,roadtypecode,severitycategory_fr,severitycategorycode,type
id,,,,,,,,,,,,,,,,,,
26D92EB4201F0112E0530A839427D7EF,vendredi / 17h-18h / décembre 2015,accident impliquant des piétons,8,2015,"[700036.0, 239831.0, 700036.0, 239831.0]",ZH,"[700036.0, 239831.0]",26D92EB4201F0112E0530A839427D7EF,0115,Feature,Fussgängerunfall,ch.astra.unfaelle-personenschaeden_fussgaenger,Accidents avec la part. de piétons,route secondaire,433,accident avec blessés graves,USV,Feature
27DD1C1B91240104E0530A839427255D,mercredi / 20h-21h / novembre 2015,dérapage ou perte de maîtrise,0,2015,"[496963.0, 118865.0, 496963.0, 118865.0]",GE,"[496963.0, 118865.0]",27DD1C1B91240104E0530A839427255D,6643,Feature,Schleuder- oder Selbstunfall,ch.astra.unfaelle-personenschaeden_motorraeder,Accidents avec la part. de motos,route principale,432,accident avec blessés légers,ULV,Feature
2A791950C04700F8E0530A83942764B5,lundi / 17h-18h / décembre 2015,accident en quittant une route,3,2015,"[571779.0, 225226.0, 571779.0, 225226.0]",BE,"[571779.0, 225226.0]",2A791950C04700F8E0530A83942764B5,0434,Feature,Abbiegeunfall,ch.astra.unfaelle-personenschaeden_fahrraeder,Accidents avec la part. de vélos,route secondaire,433,accident avec blessés légers,ULV,Feature
28A6AC491A8500C2E0530A8394273B02,vendredi / 14h-15h / décembre 2015,accident en s'engageant sur une route,4,2015,"[623267.0, 229455.0, 623267.0, 229455.0]",BE,"[623267.0, 229455.0]",28A6AC491A8500C2E0530A8394273B02,0342,Feature,Einbiegeunfall,ch.astra.unfaelle-personenschaeden_motorraeder,Accidents avec la part. de motos,route principale,432,accident avec blessés légers,ULV,Feature
E1DDFB6443CC022CE0430A839427FC92,vendredi / 17h-18h / juin 2013,accident en traversant une route,5,2013,"[703664.0, 243311.0, 703664.0, 243311.0]",ZH,"[703664.0, 243311.0]",E1DDFB6443CC022CE0430A839427FC92,0121,Feature,Überqueren der Fahrbahn,ch.astra.unfaelle-personenschaeden_alle,Accidents avec dommages corporels,route secondaire,433,accident avec blessés légers,ULV,Feature


In [10]:
for feature in df : 
    plot_feature(df, feature)
print("Done plotting - Saved plots in Resources/plots")

plotting feature accidentday_fr
plotting feature accidenttype_fr
plotting feature accidenttypecode
plotting feature accidentyear
plotting feature canton
plotting feature coordinates
->    Error
plotting feature fsocommunecode
plotting feature label
plotting feature layerName
plotting feature roadtype_fr
plotting feature roadtypecode
plotting feature severitycategory_fr
plotting feature severitycategorycode
Done plotting - Saved plots in Resources/plots


# Data analysis

1) Accidents par rapport au temps<br>
2) Corrélation nombre/type d'accident avec les endroits (Valais ivresse)<br>
3) Tracker des anomalies (fin/début d'une série d'accident) et essayer d'en trouver la cause<br>